在 Keras 中，有三种常用的模型保存与加载方法。
# 张量方式
通过调用 Model.save_weights(path)方法即可将当前的
网络参数保存到 path 文件上，代码如下：

In [ ]:
import  os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import  tensorflow as tf
from    tensorflow.keras import datasets, layers, optimizers, Sequential, metrics


def preprocess(x, y):
    """
    x is a simple image, not a batch
    """
    x = tf.cast(x, dtype=tf.float32) / 255.
    x = tf.reshape(x, [28*28])
    y = tf.cast(y, dtype=tf.int32)
    y = tf.one_hot(y, depth=10)
    return x,y


batchsz = 128
(x, y), (x_val, y_val) = datasets.mnist.load_data()
print('datasets:', x.shape, y.shape, x.min(), x.max())



db = tf.data.Dataset.from_tensor_slices((x,y))
db = db.map(preprocess).shuffle(60000).batch(batchsz)
ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
ds_val = ds_val.map(preprocess).batch(batchsz)

sample = next(iter(db))
print(sample[0].shape, sample[1].shape)


network = Sequential([layers.Dense(256, activation='relu'),
                     layers.Dense(128, activation='relu'),
                     layers.Dense(64, activation='relu'),
                     layers.Dense(32, activation='relu'),
                     layers.Dense(10)])
network.build(input_shape=(None, 28*28))
network.summary()




network.compile(optimizer=optimizers.Adam(lr=0.01),
		loss=tf.losses.CategoricalCrossentropy(from_logits=True),
		metrics=['accuracy']
	)

network.fit(db, epochs=3, validation_data=ds_val, validation_freq=2)

network.evaluate(ds_val)


In [ ]:
network.save_weights('weights.ckpt') # 保存模型的所有张量数据

上述代码将 network 模型保存到 weights.ckpt 文件上。在需要的时候，先创建好网络对象，
然后调用网络对象的 load_weights(path)方法即可将指定的模型文件中保存的张量数值写入
到当前网络参数中去，例如：

In [ ]:
# 保存模型参数到文件上
network.save_weights('weights.ckpt')
print('saved weights.')
del network # 删除网络对象

In [ ]:
# 重新创建相同的网络结构
network = Sequential([layers.Dense(256, activation='relu'),
 layers.Dense(128, activation='relu'),
 layers.Dense(64, activation='relu'),
 layers.Dense(32, activation='relu'),
 layers.Dense(10)])
network.compile(optimizer=optimizers.Adam(lr=0.01),
loss=tf.losses.CategoricalCrossentropy(from_logits=True),
metrics=['accuracy']
 )
# 从参数文件中读取数据并写入当前网络
network.load_weights('weights.ckpt')
print('loaded weights!')

这种保存与加载网络的方式最为轻量级，文件中保存的仅仅是张量参数的数值，并没有其
它额外的结构参数。但是它需要使用相同的网络结构才能够正确恢复网络状态，因此一般
在拥有网络源文件的情况下使用

# 网络方式
介绍一种不需要网络源文件，仅仅需要模型参数文件即可恢复出网络模型的方
法。通过 Model.save(path)函数可以将模型的结构以及模型的参数保存到 path 文件上，在不
需要网络源文件的条件下，通过 keras.models.load_model(path)即可恢复网络结构和网络参
数。

In [ ]:
# 保存模型结构与模型参数到文件
network.save('model.h5')
print('saved total model.')
del network # 删除网络对象

In [ ]:
from tensorflow import keras

# 从文件恢复网络结构与网络参数
network = keras.models.load_model('model.h5')

model.h5 文件除了保存了模型参数外，还应保存了网络结构信息，不需要提前
创建模型即可直接从文件中恢复出网络 network 对象。

# SavedModel方式

In [ ]:
# 保存模型结构与模型参数到文件
tf.saved_model.save(network, 'model-savedmodel')
print('saving savedmodel.')
del network # 删除网络对象

用户无需关心文件的保存格式，只需要通过 tf.saved_model.load 函数即可恢复出模型
对象，我们在恢复出模型实例后，完成测试准确率的计算，实现如下

In [ ]:
print('load savedmodel from file.')
# 从文件恢复网络结构与网络参数
network = tf.saved_model.load('model-savedmodel')

In [ ]:
# 准确率计量器
acc_meter = metrics.CategoricalAccuracy()
for x,y in ds_val: # 遍历测试集
    pred = network(x) # 前向计算
    acc_meter.update_state(y_true=y, y_pred=pred) # 更新准确率统计
# 打印准确率
print("Test Accuracy:%f" % acc_meter.result())